In [ ]:
import sqlite3
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel
from dotenv import load_dotenv
import os
import json

load_dotenv()
cohere_token = os.getenv("cohere_key")

# Define the agent
model = LiteLLMModel(
    model_id="command-r",
    api_key=cohere_token,
)
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

# Define the function to choose the best algorithm
def choose_best_algorithm(train_db_file="train_mTSP.sqlite3", test_db_file="test_mTSP.sqlite3"):

    conn_train = sqlite3.connect(train_db_file)
    cursor_train = conn_train.cursor()

    # Select instances from the training database
    cursor_train.execute("""
        SELECT instance_id, nr_cities, nr_salesmen, average_distance, stddev_distance, density, salesmen_ratio, 
               bounding_box_area, aspect_ratio, spread, cluster_compactness, mst_total_length, entropy_distance_matrix 
        FROM instances 
    """)
    train_instances = cursor_train.fetchall()
    train_instances_with_columns = [
        {
            "instance_id": row[0], 
            "nr_cities": row[1], 
            "nr_salesmen": row[2],
            "average_distance": row[3],
            "stddev_distance": row[4],
            "density": row[5],
            "salesmen_ratio": row[6],
            "bounding_box_area": row[7],
            "aspect_ratio": row[8],
            "spread": row[9],
            "cluster_compactness": row[10],
            "mst_total_length": row[11],
            "entropy_distance_matrix": row[12]
        } for row in train_instances
    ]

    # Select algorithms from the training database
    cursor_train.execute("""
        SELECT instance_id, strategy, total_cost, normalized_cost, time_taken, distance_gap, efficiency 
        FROM algorithms 
        WHERE instance_id IN (SELECT instance_id FROM instances)
    """)
    train_algorithms_data = cursor_train.fetchall()
    train_algorithms_with_columns = [
        {
            "instance_id": row[0],
            "strategy": row[1],
            "total_cost": row[2],
            "normalized_cost": row[3],
            "time_taken": row[4],
            "distance_gap": row[5],
            "efficiency": row[6],
        }
        for row in train_algorithms_data
    ]


    conn_train.close()
    conn_test = sqlite3.connect(test_db_file)
    cursor_test = conn_test.cursor()

    # Select instances from the test database
    cursor_test.execute("""
        SELECT instance_id, nr_cities, nr_salesmen, average_distance, stddev_distance, density, salesmen_ratio, 
               bounding_box_area, aspect_ratio, spread, cluster_compactness, mst_total_length, entropy_distance_matrix 
        FROM instances 
    """)
    test_instances = cursor_test.fetchall()
    test_instances_with_columns = [
        {
            "instance_id": row[0], 
            "nr_cities": row[1], 
            "nr_salesmen": row[2],
            "average_distance": row[3],
            "stddev_distance": row[4],
            "density": row[5],
            "salesmen_ratio": row[6],
            "bounding_box_area": row[7],
            "aspect_ratio": row[8],
            "spread": row[9],
            "cluster_compactness": row[10],
            "mst_total_length": row[11],
            "entropy_distance_matrix": row[12]
        } for row in test_instances
    ]

    conn_test.close()

    # Prepare the data for the agent
    data = {
        "train_instances": train_instances_with_columns,
        "train_algorithms": train_algorithms_with_columns,
        "test_instances": test_instances_with_columns,
    }
    data_str = json.dumps(data, indent=4)

    # Task for the agent
    task = f"""
    You are an AI agent tasked with selecting the best algorithm for solving each instance of the mTSP problem.
    Below is the data from the training instances and their results for different strategies:

    {data_str}

    Your task is to:
    1. Analyze the performance of each strategy (Greedy, Branch and Cut, Ant Colony, KMeans-Greedy) on the training instances.
    2. Based on this analysis, recommend the best strategy for each test instance.
    3. Justify your recommendations based on the training data.
    4. Consider the following performance metrics: total cost, normalized_cost, time taken, efficiency, and distance gap. The total cost is the most important metric, followed by normalized_cost and efficiency. Time taken is less important, but it should be considered for strategies that are computationally expensive.
    5. Take into account that Branch and Cut and Ant Colony are more computationally expensive, while Greedy and KMeans-Greedy are faster but may not always yield the best results.
    6. Branch and Cut and Ant Colony have a 100 seconds time limit, so they may output a weaker solution if the time limit is reached or even no solution at all.

    Output your recommendations in the following format:
    [
    {{ "instance_id": <test_instance_id>, "recommended_strategy": <strategy>, "justification": <reason> }}
    ]   
    """

    response = agent.run(task)
    print(response)

choose_best_algorithm(train_db_file="train_mTSP.sqlite3", test_db_file="test_mTSP.sqlite3")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an AI agent tasked with selecting the best algorithm for solving each instance of the mTSP problem.     │
│     Below is the data from the training instances and their results for different strategies:                   │
│                                                                                                                 │
│     {                                                                                                           │
│     "train_instances": [                                                                                        │
│         {                                                                                                       │
│             "instance_id": 1,                                                                                   │
│             "nr_cities": 15,                                                                                    │
│             "nr_salesmen": 3,                                                                                   │
│             "average_distance": 6822.850650201972,                                                              │
│             "stddev_distance": 6766.0401414006865,                                                              │
│             "density": 0.7725147928217813,                                                                      │
│             "salesmen_ratio": 0.2,                                                                              │
│             "bounding_box_area": 8832.0,                                                                        │
│             "aspect_ratio": 0.9583333333333334,                                                                 │
│             "spread": 0.016425891691716515,                                                                     │
│             "cluster_compactness": 454.8567100134648,                                                           │
│             "mst_total_length": 309.276067867573,                                                               │
│             "entropy_distance_matrix": 2.7959530609379706                                                       │
│         },                                                                                                      │
│         {                                                                                                       │
│             "instance_id": 2,                                                                                   │
│             "nr_cities": 15,                                                                                    │
│             "nr_salesmen": 3,                                                                                   │
│             "average_distance": 6862.288677204529,                                                              │
│             "stddev_distance": 6805.155847429309,                                                               │
│             "density": 0.7935116416748993,                                                                      │
│             "salesmen_ratio": 0.2,                                                                              │
│             "bounding_box_area": 8648.0,                                                                        │
│             "aspect_ratio": 1.0217391304347827,                                                                 │
│             "spread": 0.018664619197186007,                                                                     │
│             "cluster_compactness": 457.4859118136353,                                                           │
│             "mst_total_length": 310.72208166143344,   

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            To determine the best strategy for each test instance, I analyzed the performance of Greedy, Branch and
Cut, and Ant Colony algorithms on the training instances. Here are the key findings and recommendations:

**Analysis of Training Instances:**

1. **Branch and Cut**: 
   - Achieves the lowest total and normalized costs in most cases.
   - Has a significantly higher distance gap, indicating it finds solutions closer to the optimal.
   - However, it takes considerably more time, especially for larger instances (e.g., instances 11-15 and 21-30).
   - Efficiency decreases as instance size increases.

2. **Ant Colony**:
   - Generally has lower costs than Greedy but higher than Branch and Cut.
   - Much faster than Branch and Cut, with time taken being relatively consistent across instance sizes.
   - Efficiency is moderate, balancing solution quality and computation time.

3. **Greedy**:
   - Fastest algorithm with near-zero time taken.
   - Highest total and normalized costs, and the largest distance gap.
   - Efficiency is extremely high due to its speed, but solution quality is the lowest.

**Recommendations for Test Instances:**

```json
[
    {
        "instance_id": 1,
        "recommended_strategy": "Branch and Cut",
        "justification": "Small instance size (10 cities) allows Branch and Cut to find optimal solutions 
efficiently."
    },
    {
        "instance_id": 2,
        "recommended_strategy": "Branch and Cut",
        "justification": "Small instance size (10 cities) allows Branch and Cut to find optimal solutions 
efficiently."
    },
    {
        "instance_id": 3,
        "recommended_strategy": "Branch and Cut",
        "justification": "Small instance size (10 cities) allows Branch and Cut to find optimal solutions 
efficiently."
    },
    {
        "instance_id": 4,
        "recommended_strategy": "Branch and Cut",
        "justification": "Small instance size (10 cities) allows Branch and Cut to find optimal solutions 
efficiently."
    },
    {
        "instance_id": 5,
        "recommended_strategy": "Branch and Cut",
        "justification": "Small instance size (10 cities) allows Branch and Cut to find optimal solutions 
efficiently."
    },
    {
        "instance_id": 6,
        "recommended_strategy": "Branch and Cut",
        "justification": "Small instance size (10 cities) allows Branch and Cut to find optimal solutions 
efficiently."
    },
    {
        "instance_id": 7,
        "recommended_strategy": "Branch and Cut",
        "justification": "Small instance size (10 cities) allows Branch and Cut to find optimal solutions 
efficiently."
    },
    {
        "instance_id": 8,
        "recommended_strategy": "Branch and Cut",
        "justification": "Small instance size (10 cities) allows Branch and Cut to find optimal solutions 
efficiently."
    },
    {
        "instance_id": 9,
        "recommended_strategy": "Branch and Cut",
        "justification": "Small instance size (10 cities) allows Branch and Cut to find optimal solutions 
efficiently."
    },
    {
        "instance_id": 10,
        "recommended_strategy": "Ant Colony",
        "justification": "Larger instance size (10 cities) and higher complexity make Ant Colony a better choice 
than Greedy, while Branch and Cut becomes inefficient."
    },
    {
        "instance_id": 11,
        "recommended_strategy": "Ant Colony",
        "justification": "Larger instance size (15 cities) makes Branch and Cut too slow; Ant Colony offers a 
better balance of solution quality and speed."
    },
    {
        "instance_id": 12,
        "recommended_strategy": "Ant Colony",
        "justification": "Larger instance size (15 cities) makes Branch and Cut too slow; Ant Colony offers a 
better balance of solution quality and speed."
    },
    {
        "instance_i

[Step 1: Duration 54.51 seconds| Input tokens: 82,867 | Output tokens: 2,010]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 15 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



Error in generating model output:
litellm.APIConnectionError: Cohere_chatException - {"message":"You are using a Trial key, which is limited to 10 
API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate 
limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at 
support@cohere.com with any questions"}

[Step 15: Duration 0.25 seconds| Input tokens: 87,600 | Output tokens: 7,813]

AgentGenerationError: Error in generating model output:
litellm.APIConnectionError: Cohere_chatException - {"message":"You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}